In [19]:
import torch
torch.backends.mps.is_available()

True

In [3]:
pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu


Looking in indexes: https://download.pytorch.org/whl/nightly/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 MB 6.6 MB/s  0:00:12m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.8 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.5/737.5 kB 1.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [torchaudio]3 [torchvision]
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("MPS available:", torch.backends.mps.is_available())
print("MPS built:", torch.backends.mps.is_built())

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.randn(1000, 1000, device=mps_device)
    y = x @ x
    print("MPS works!")
else:
    print("MPS not available.")

CUDA available: False
MPS available: True
MPS built: True
MPS works!


In [6]:
pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 6.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [timm]4/5 [timm]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
import timm
from pathlib import Path
import os
import json
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from tqdm import tqdm
import time 
import random

In [22]:
torch.manual_seed(42)
torch.mps.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [23]:
# setting primary compute as gpu
device = torch.device("mps")

    

In [50]:
import shutil

base_dir = "Downloads/lung_colon_image_set"
sources = ["colon_image_sets", "lung_image_sets"]

for src in sources:
    src_path = os.path.join(base_dir, src)
    for item in os.listdir(src_path):
        item_path = os.path.join(src_path, item)
        if os.path.isdir(item_path):
            shutil.move(item_path, base_dir)


In [51]:
os.listdir("Downloads/lung_colon_image_set")

['lung_aca',
 'colon_n',
 'lung_image_sets',
 'colon_aca',
 'colon_image_sets',
 'lung_n',
 'lung_scc']

In [112]:
class Lc25000Data(Dataset):
    def __init__(self, dir, transform = None):

        self.dir = Path(dir)
        self.transform = transform

        self.classes = ['colon_aca', 'colon_n', 'lung_aca', 'lung_n', 'lung_scc']

        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}


        self.images = []
        self.labels = []

        
        for class_name in self.classes:
            class_dir = self.dir / class_name
            if class_dir.exists():
                for img_path in class_dir.glob('*.jpeg'):
                    self.images.append(img_path)
                    self.labels.append(self.class_to_idx[class_name])


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')

        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [113]:
# defining image transformations

# mean and std values are taken for ImageNet - can be changed for other models
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale = (0.8,1.0)),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomRotation(degrees = 15),
    transforms.ColorJitter(brightness =  0.2, contrast = 0.2, saturation = 0.2, hue = 0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
                         ])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         ])

In [114]:
data = Path("Downloads/lung_colon_image_set")

In [115]:
train_dataset = Lc25000Data(data, transform = train_transform)


In [116]:
print(f"Total training samples: {len(train_dataset)}")


Total training samples: 25000


In [117]:
total_samples = len(train_dataset)
train_size = int(0.8 * total_samples)
val_size = total_samples - train_size

indices = torch.randperm(total_samples).tolist()
train_indices = indices[:train_size]
val_indices = indices[train_size:]

print(f"Training samples: {len(train_indices)}")
print(f"Validation samples: {len(val_indices)}")

Training samples: 20000
Validation samples: 5000


In [118]:
from torch.utils.data import Subset

train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(train_dataset, val_indices)

In [119]:
batch_size = 32
num_workers = 0

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=
                        num_workers)

In [120]:
# ResNet Model Implementation

class ResNetModel(nn.Module):
    def __init__(self, num_classes=5, pretrained = True):
        super(ResNetModel, self).__init__()

        if pretrained:
            weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2
            self.backbone = torchvision.models.resnet50(weights=weights)

        else:
            self.backbone = torchvision.models.resnet50(weights=None)

        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_features, num_classes)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.dropout(x)
        x = self.backbone.fc(x)

        return x

        

In [64]:
resnet_model = ResNetModel(num_classes = 5, pretrained = True).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /Users/sharathsr/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████████████████████████████████| 97.8M/97.8M [00:15<00:00, 6.79MB/s]


In [121]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"res net")
print(f"total parameters: {count_params(resnet_model)}")


res net
total parameters: 23518277


In [132]:
def train_model(model, train_loader, val_loader, num_epochs = 25, lr = 1e-4, patience = 7):


    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', 
                                                     factor=0.5, patience=3)

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    best_val_acc = 0.0
    patience_counter = 0

    best_model_state = None

    is_parallel = isinstance(model, nn.DataParallel)

    for epoch in range(num_epochs):
        start_time = time.time()

        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_samples = 0

        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} [Train]')

        
        for batch_idx, (images, labels) in enumerate(train_pbar):
            
            images, labels = images.to(device), labels.to(device)
            
            
            optimizer.zero_grad()

            
            
            outputs = model(images)

            

            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            var1, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_preds += (predicted == labels).sum().item()

            train_pbar.set_postfix({
                'Loss': f'{running_loss/(batch_idx+1):.4f}',
                'Acc': f'{100.*correct_preds/total_samples:.2f}%'
            })


        train_loss = running_loss/len(train_loader)
        train_acc = correct_preds / total_samples


        # Validation

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc = f'Epoch {epoch + 1}/{num_epochs} [Val]')

            for images, labels in val_pbar:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                var1, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                val_pbar.set_postfix({
                    'Loss': f'{val_loss/len(val_loader):.4f}',
                    'Acc': f'{100.*correct/total:.2f}%'
                })

        val_loss /= len(val_loader)
        val_acc = correct / total

        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_acc)

        new_lr = optimizer.param_groups[0]['lr']


        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        epoch_time = time.time() - start_time

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        print(f'  Learning Rate: {new_lr:.2e}')

        if val_acc > 0.99:
            print("Very high validation accuracy - possible overfitting!")
        if val_loss < 0.01:
            print("Very low validation loss - possible data leakage!")
        if len(val_accuracies) > 1 and val_acc < val_accuracies[-2] - 0.05:
            print("Validation accuracy dropped significantly!")

        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            
            best_model_state = model.module.state_dict().copy() if is_parallel else model.state_dict().copy()
            patience_counter = 0
            print(f'New best validation accuracy: {best_val_acc:.4f}')
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f'Early stopping triggered after {epoch+1} epochs')
            break

        
    if best_model_state is not None:
        if is_parallel:
            model.module.load_state_dict(best_model_state)
        else:
            model.load_state_dict(best_model_state)
        print(f'Loaded best model with validation accuracy: {best_val_acc:.4f}')
    
    return {
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'val_losses': val_losses,
        'val_accuracies': val_accuracies,
        'best_val_acc': best_val_acc
    }
    

In [133]:
def save_model(model, history, model_name, save_dir = "models"):
    os.makedirs(save_dir, exist_ok = True)

    model_path = os.path.join(save_dir, f"{model_name}_model.pth")

    torch.save(model.state_dict(), model_path)

    

    return model_path
    

    

In [134]:
print("Starting ResNet Training...")

print("=" * 60)

class_names = ['colon_aca', 'colon_n', 'lung_aca', 'lung_n', 'lung_scc']

resnet_model = ResNetModel(num_classes=5, pretrained=True).to(device)

resnet_history = train_model(
    model=resnet_model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=10,
    lr=1e-4,
    patience=7
)

save_model(resnet_model, resnet_history, "ResNet50")





Starting ResNet Training...


Epoch 1/10 [Train]: 100%|█| 625/625 [08:39<00:00,  1.20it/s, Loss=0.1944, Acc=93
Epoch 1/10 [Val]: 100%|█| 157/157 [00:59<00:00,  2.64it/s, Loss=0.0327, Acc=99.0


Epoch 1/10:
  Train Loss: 0.1944, Train Acc: 0.9369
  Val Loss: 0.0327, Val Acc: 0.9904
  Learning Rate: 1.00e-04
Very high validation accuracy - possible overfitting!
New best validation accuracy: 0.9904


Epoch 2/10 [Train]: 100%|█| 625/625 [08:42<00:00,  1.20it/s, Loss=0.0476, Acc=98
Epoch 2/10 [Val]: 100%|█| 157/157 [00:59<00:00,  2.62it/s, Loss=0.0179, Acc=99.5


Epoch 2/10:
  Train Loss: 0.0476, Train Acc: 0.9841
  Val Loss: 0.0179, Val Acc: 0.9950
  Learning Rate: 1.00e-04
Very high validation accuracy - possible overfitting!
New best validation accuracy: 0.9950


...

Epoch 9/10 [Train]: 100%|█| 625/625 [08:43<00:00,  1.19it/s, Loss=0.0127, Acc=99
Epoch 9/10 [Val]: 100%|█| 157/157 [00:59<00:00,  2.63it/s, Loss=0.0040, Acc=99.8


Epoch 9/10:
  Train Loss: 0.0127, Train Acc: 0.9965
  Val Loss: 0.0040, Val Acc: 0.9988
  Learning Rate: 1.00e-04
Very high validation accuracy - possible overfitting!
Very low validation loss - possible data leakage!


Epoch 10/10 [Train]: 100%|█| 625/625 [08:44<00:00,  1.19it/s, Loss=0.0110, Acc=9
Epoch 10/10 [Val]: 100%|█| 157/157 [00:59<00:00,  2.63it/s, Loss=0.0078, Acc=99.


Epoch 10/10:
  Train Loss: 0.0110, Train Acc: 0.9968
  Val Loss: 0.0078, Val Acc: 0.9968
  Learning Rate: 1.00e-04
Very high validation accuracy - possible overfitting!
Very low validation loss - possible data leakage!
Loaded best model with validation accuracy: 0.9990


'models/ResNet50_model.pth'

In [136]:
os.listdir('models')

['ResNet50_model.pth']

NameError: name 'git' is not defined